# HDF5 Database in pySwap

below is a quick example on how to use the HDF5 database format within the pySWAP. HDF5 (Hierarchical Data Format version 5) is a versatile data model that can represent complex data objects and a wide variety of metadata. It is widely used for storing large amounts of data efficiently and is particularly useful in scientific computing. It can be used for partial retrieval as well, potentially limiting the number of data stored.

In this section we:
- Create and manage HDF5 files with pySwap.
- Store and retrieve data in HDF5 format.


In [ ]:
import pyswap as ps
from pyswap import testcase

# get the Model object and run it
model = testcase.get("hupselbrook")
result = model.run("./")

# now, let's create an inteface object and save the model and the result
f = ps.HDF5(filename="data.h5")
f.save_model(model=model, result=result)

## Loading and Modifying Models

Above you can see that the model was saved to the .h5 file. If the file already exists (for example one with your other projects),
you will get a warning message and the program will skip creation of additional elements in the database.

Now we can:
- List the models stored in the HDF5 file.
- Load a specific model from the HDF5 file.
- Create a copy of the loaded model and modify its parameters for another run.

In [ ]:
f.list_models

In [ ]:
db = f.load("pySWAP test - hupselbrook", load_results=True)

In [ ]:
ml2 = db["base"][0].model_copy(deep=True)

In [ ]:
# We modify the initial groundwater level:
soil_moisture_alt = ml2.soilmoisture.model_copy(update={"gwli": -74})

In [ ]:
# then we ammend the main model object, remembering to change the version name:
ml_gwliup = ml2.model_copy(
    update={"soilmoisture": soil_moisture_alt, "version": "gwli_up"}
)

In [ ]:
# And run the model
result2 = ml_gwliup.run("./")

In [ ]:
# Savin the model version to the database
f.save_model(model=ml_gwliup)

f.list_models

## Visualise the results

Now you can observe the results by plotting the main output pandas.DataFrame object, like you normally would.

In [ ]:
result2.output["GWL"].plot()